<a href="https://colab.research.google.com/github/JS19563255/2022bioinfo1_project/blob/main/week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

환경 구축

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'colab-biolab' already exists and is not an empty directory.
./
./root/
./root/.tmux.conf
./root/.profile
./root/.bin.priority/
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.bin.priority/pip3
./root/.condarc
./root/.bashrc.biolab
./root/.vimrc
PREFIX=/root/conda
Unpacking payload ...
Solving environment: - \ | / - done

# All requested packages already installed.

installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /root/conda
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - 

파일 확인

In [3]:
!ls -al /content/drive/MyDrive/binfo1-datapack1/

total 7136706
-r-------- 1 root root 1370036258 May  6  2021 CLIP-35L33G.bam
-r-------- 1 root root    3118336 May  6  2021 CLIP-35L33G.bam.bai
-r-------- 1 root root  883334756 May  5  2021 gencode.gtf
-r-------- 1 root root 1003658801 May  6  2021 RNA-control.bam
-r-------- 1 root root    2276104 May  6  2021 RNA-control.bam.bai
-r-------- 1 root root 1260991122 May  6  2021 RNA-siLin28a.bam
-r-------- 1 root root    2710744 May  6  2021 RNA-siLin28a.bam.bai
-r-------- 1 root root  981684502 May  6  2021 RNA-siLuc.bam
-r-------- 1 root root    2606104 May  6  2021 RNA-siLuc.bam.bai
-r-------- 1 root root  737352902 May  6  2021 RPF-siLin28a.bam
-r-------- 1 root root    2482768 May  6  2021 RPF-siLin28a.bam.bai
-r-------- 1 root root 1055123479 May  6  2021 RPF-siLuc.bam
-r-------- 1 root root    2605912 May  6  2021 RPF-siLuc.bam.bai


In [4]:
!md5sum drive/MyDrive/binfo1-datapack1/*
!ls -al drive/MyDrive/binfo*

140aaf30bcb9276cc716f8699f04ddd6  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam.bai
d54d428f5872030c4ffc43058709b896  drive/MyDrive/binfo1-datapack1/gencode.gtf
328883a73d507eafbf5b60bd6b906201  drive/MyDrive/binfo1-datapack1/RNA-control.bam
02073818e2f398a73c3b76e5169de1ca  drive/MyDrive/binfo1-datapack1/RNA-control.bam.bai
b09550d09d6c2a4ce27f0226f426fdb1  drive/MyDrive/binfo1-datapack1/RNA-siLin28a.bam
fef112c727244060ea62d3f2564a07f6  drive/MyDrive/binfo1-datapack1/RNA-siLin28a.bam.bai
28bbd0c47d725669340c784f1b772c01  drive/MyDrive/binfo1-datapack1/RNA-siLuc.bam
43590fdc4d81905c0432e0d1cb8cfd5b  drive/MyDrive/binfo1-datapack1/RNA-siLuc.bam.bai
5c08a9297307bc83259e658c4474f0cc  drive/MyDrive/binfo1-datapack1/RPF-siLin28a.bam
a1bb3e29be412dfd7fd8d16b1b1acc4c  drive/MyDrive/binfo1-datapack1/RPF-siLin28a.bam.bai
f2eebf50943024d0116c9cd3e744c707  drive/MyDrive/binfo1-datapack1/RPF-siLuc.bam
dc24f69e8f571f

In [5]:
!conda install -y bedtools bioawk

%cd /content/drive/MyDrive/binfo1-datapack1/

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.11.0
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/conda/envs/lab

  added / updated specs:
    - bedtools
    - bioawk


The following NEW packages will be INSTALLED:

  bioawk             bioconda/linux-64::bioawk-1.0-h7132678_7


Preparing transaction: \ done
Verifying transaction: / done
Executing transaction: \ done
/content/drive/.shortcut-targets-by-id/1Y190lbO42VEzjjQkgoxNf_EcZJlHyZa1/binfo1-datapack1


우선 annotation에서 start codon이 포함된 exon만 추립니다. Gene 한 개에 isoform이 여러 개 있는 경우 모두 쓰면 read가 너무 부풀려질 수 있기 때문에 우선 transcript support level이 1인 것만 추려서 사용합니다. 여전히 TSL 1만 쓰더라도 부풀려질 가능성은 있기 때문에 제대로 하자면 APPRIS나 RNA-seq 데이터 기준으로 하나만 선정하는 것이 좋습니다만, 일단 여기서는 간단히..

그리고, - strand가 들어가면 좀 계산이 복잡해지니까 또 간단하게 하기 위해서 + strand만 고릅니다. 이번엔 일단 무조건 간단하게 해 봅시다~

In [6]:
%cd /content/drive/MyDrive/binfo1-datapack1/

/content/drive/.shortcut-targets-by-id/1Y190lbO42VEzjjQkgoxNf_EcZJlHyZa1/binfo1-datapack1


In [7]:
!grep '	start_codon	.*	+	.*transcript_support_level "1"' gencode.gtf | \
 sed -e 's/	[^	]*transcript_id "\([^"]*\)".*$/	\1/g' > gencode-start.gtf
!head gencode-start.gtf

/bin/bash: gencode-start.gtf: Read-only file system
head: cannot open 'gencode-start.gtf' for reading: No such file or directory


start codon가 들어가 있는 exon들을 고르기 위해서 미리 + strand에 있는 exon를 추립니다.. start codon 쪽에서 TSL 기준으로 골랐기 때문에 여기서는 고르지 않아도 됩니다. 물론 좀 느려지기는 하지만...

In [8]:
!grep '	exon	.*	+	' gencode.gtf | \
 sed -e 's/	[^	]*transcript_id "\([^"]*\)".*$/	\1/g' > gencode-plusexon.gtf
!head gencode-plusexon.gtf

/bin/bash: gencode-plusexon.gtf: Read-only file system
head: cannot open 'gencode-plusexon.gtf' for reading: No such file or directory


그 다음엔 exon 리스트와 start codon 리스트를 겹치고, transcript가 같은 것만 살려둡니다. 기왕 좀 다루기 쉽도록 출력은 bed로 하면서 좌표계를 바꿉니다. 결과는 bedtools에서 빠르게 다룰 수 있게 sort해 둡니다.

In [9]:
!bedtools intersect -a gencode-start.gtf -b gencode-plusexon.gtf -wa -wb | \
 awk -F'	' -v OFS='	' '$9 == $18 { print $10, $13-1, $14, $18, $4-1, $16; }' | \
 sort -k1,1 -k2,3n -k4,4 > gencode-exons-containing-startcodon.bed
!head gencode-exons-containing-startcodon.bed; tail gencode-exons-containing-startcodon.bed

/bin/bash: gencode-exons-containing-startcodon.bed: Read-only file system
Error: Unable to open file gencode-start.gtf. Exiting.
head: cannot open 'gencode-exons-containing-startcodon.bed' for reading: No such file or directory
tail: cannot open 'gencode-exons-containing-startcodon.bed' for reading: No such file or directory


컬럼 4번에 transcript id, 5번에 start codon 위치를 넣었는데, start codon이 같은 것들이 여러개 눈에 띕니다. 아주 찝찝하지만, 우선 이번은 간단하게 할 것이기 때문에 눈을 질끈 감고 넘어가 봅시다.

이제 alignment를 준비합니다. + strand alignment에서 read length가 짧은 것도 미리 필터링 합니다. RPF는 보통 25nt 이상이기 때문에 그 이하는 거의 확실히 rRNA, tRNA 등에서 오염된 것들입니다. SAM 파일 전체를 필터링해서 다시 인코딩하기 때문에 시간이 좀 걸립니다.

In [10]:
!(samtools view -H RPF-siLuc.bam; \
  samtools view -F20 RPF-siLuc.bam | \
  bioawk -c sam '{ if (length($seq) >= 25) print $0; }') | \
 samtools view -b -o filtered-RPF-siLuc.bam

[E::hts_open_format] Failed to open file "filtered-RPF-siLuc.bam" : Read-only file system
samtools view: failed to open "filtered-RPF-siLuc.bam" for writing: Read-only file system


큰 일을 했으니 용량을 한 번 봅니다.

In [11]:
!ls -al *RPF-siLuc.bam

-r-------- 1 root root 1055123479 May  6  2021 RPF-siLuc.bam


자 이제 read의 5' end 위치를 셉니다. bedtools의 `genomecov`를 쓰면 아주 편리합니다.

In [12]:
!bedtools genomecov -ibam filtered-RPF-siLuc.bam -bg -5 > fivepcounts-RPF-siLuc.bed
!head fivepcounts-RPF-siLuc.bed

/bin/bash: fivepcounts-RPF-siLuc.bed: Read-only file system
head: cannot open 'fivepcounts-RPF-siLuc.bed' for reading: No such file or directory


그렇게 match 된 것들 중에 앞에서 찾은 start codon을 포함한 exon과 겹치는 것만 골라냅니다.

In [13]:
!bedtools intersect -a fivepcounts-RPF-siLuc.bed -b gencode-exons-containing-startcodon.bed \
 -wa -wb -nonamecheck > fivepcounts-filtered-RPF-siLuc.txt
!head fivepcounts-filtered-RPF-siLuc.txt

/bin/bash: fivepcounts-filtered-RPF-siLuc.txt: Read-only file system
head: cannot open 'fivepcounts-filtered-RPF-siLuc.txt' for reading: No such file or directory


모든 준비가 끝났습니다. 이제 Figure S5A 도전해 보세요!

* stop codon은 빼고 start codon 만입니다.
* 논문에서는 처리가 어려운 부분까지 모두 포함됐고, read를 중첩해서 세지 않았기 때문에, 여기서 세는 것과는 수치는 다를 수 있습니다.
* `pandas`로 한꺼번에 데이터를 모두 불러들이면 메모리가 부족할 수도 있습니다. 그런 경우에는 chunk를 쓰거나, 줄 단위로 처리해 보세요.